In [101]:
import math
import pandas as pd
import numpy

In [102]:
# There are three scenarios for (4,4) currently.
# 1. Stake every Epoch: User claims vested ohm tokens from bond and stakes before following epoch
# 2. Stake daily: User claims vested ohm tokens and stakes once a day
# 3. Stake halfway: User claims vest ohm tokens and stakes half way through the vesting period of 5 days
# In all calculations, gas fees for claim and autostake will be considered
# The simulator function will take a selected scenario as argument

#scenarios = ['stakeEveryEpoch','stakeDaily','stakeHalfway'] # list of scenarios
#selectedScenario = scenarios[0] # Selected scenario


# Variable declarations: Eth and Ohm prices
priceofETH = 3200
initOhmValue = initialOhms * ohmPrice

# Variable declarations: Fees
# miscFee = 823373 * ((gwei*priceofETH)/(10**9))
gwei = 30
stakingGasFee = 179123*((gwei*priceofETH)/(10**9)) 
unstakingGasFee = 89654*((gwei*priceofETH)/(10**9))
swappingGasFee = 225748*((gwei*priceofETH)/(10**9))+((0.3/100)*initOhmValue)
claimGasFee = 80209*((gwei*priceofETH)/(10**9))
bondingGasFee = 258057*((gwei*priceofETH)/(10**9))

#


bondingSimulationResults = [] # Instantiate an array to hold the results from the simulation

# Data frame to hold all required data point. Data required would be Epochs since rebase are distributed every Epoch
bondingSimulationResult_ohmGrowth_df = pd.DataFrame(np.arange(1096), columns=['Epochs'])  # In this case let's consider 1096 Epochs which is 365 days
bondingSimulationResult_ohmGrowth_df['Days'] = bondingSimulationResult_ohmGrowth_df.Epochs / 3  # There are 3 Epochs per day so divide by 3 to get Days

def bondingSimulation(discountedOHMPrice, amountUSDBonded, priceofETH, gasPrice, bondROI, nextEpochRebase,selectedScenario):
    
# Since there will be multiple scenarios (Possibly growing over time, we need to utilize if statements for scenarios

# First Scenario is staking before every epoch

#========================================================================================
# Calculate the rebase rate and Current APY (next epoch rebase pulled from hippo data source)

    rewardRate = round(nextEpochRebase/100, 4)
    rebaseConst = 1 + rewardRate  # calculate a constant for use in APY calculation
    currentAPY = (rebaseConst)**(1095)-1  # current APY equation
    currentAPY_P = (currentAPY)*100  # convert to %
    
#=========================================================================================    
# Case selection
#=========================================================================================

# STAKING BEFORE EVERY EPOCH
# ========================================================================================
    if selectedScenario =='stakeEveryEpoch':

# (3,3)
        stakingRate = (rebaseConst)**15 - 1  # staking reward rate
        stakingRate_P = round(stakingRate * 100, 4) # staking reward rate in percentage
        stakingOhmsGained = round(amountUSDBonded*stakingRate/discountedOHMPrice, 4) # Ohms gained from staking scenario
        vestedBonus = ((rebaseConst)*(1-(rebaseConst)**15) /(1-(rebaseConst))/15)-1  # ohms gained as a percentage
        vestedBonus_P = round(vestedBonus * 100, 2) # extra ohms gained in percentage
# ================================================================================

# (4,4)
        bondingRate = (round(bondROI/100, 4))+vestedBonus  # bonding reward rate
        bondingRate_P = round(bondingRate * 100, 4) # bonding reward rate in percentage
        bondingOhmsGained = (amountUSDBonded*bondingRate / discountedOHMPrice)  # ohms gained from bonding
# ================================================================================

# Rates comparison
        rateDiff = bondingRate - stakingRate
        rateDiff_P = rateDiff * 100
        minBondDiscount = stakingRate_P - vestedBonus_P
# ================================================================================

#ROI and APY
        effectiveRebaseRate = 10**(math.log10(1+bondingRate)/15)-1
        bondingAPY = (1+bondingRate)**(365/5)-1
        bondingAPY_P = bondingAPY * 100
        apyMultiplier = bondingAPY/currentAPY

        return rewardRate, rebaseConst, currentAPY, currentAPY_P, stakingRate, stakingRate_P, \
        stakingOhmsGained, vestedBonus, vestedBonus_P, bondingRate, bondingRate_P,\
        bondingOhmsGained, rateDiff, rateDiff_P, minBondDiscount, effectiveRebaseRate, bondingAPY, bondingAPY_P, apyMultiplier
    
    else:
        return ('Consider using gas consideration')

In [103]:
update_Incooom(431.85, 8000, 2400, 40, 5.58, 0.40,selectedScenario)

(0.004,
 1.004,
 78.14354983991076,
 7814.354983991077,
 0.06170947253567838,
 6.1709,
 1.1432,
 0.03260517376368388,
 3.26,
 0.08840517376368388,
 8.8405,
 1.6377014938276508,
 0.026695701228005506,
 2.6695701228005504,
 2.9109,
 0.005663542992631498,
 483.96860019893376,
 48396.86001989338,
 6.193327551543523)